In [1]:
from Okezone import Okezone
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [2]:
obj = Okezone()

In [3]:
end_date = date(2018, 8, 12)
start_date = date(2018, 1, 1)
init_page = 1

In [4]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, 0, single_date.strftime("%Y/%m/%d"))
    init_page = 1

2018-01-01
page  1
https://index.okezone.com/bydate/index/2018/01/01/0/
memasukkan berita id  1838585
Insert berita  Stasiun Cirebon Mulai Dipadati Penumpang Arus Balik dari Jakarta   
masuk
memasukkan berita id  1838586
Insert berita  Sang Ksatria Bergitar Ungkap Harapan di Tahun 2018
masuk
memasukkan berita id  1838566
Insert berita  Maret 2018, Armada Bakal Gelar Konser di Malaysia
masuk
memasukkan berita id  1838494
Insert berita  Kevin Magnussen Tak Peduli Sering Dikritik Lakoni Balapan Agresif
masuk
memasukkan berita id  1838584
Insert berita  Dekorasi ala Hutan Warnai Penampilan Yovie & Nuno di Penghujung 2017
masuk
memasukkan berita id  1838583
Insert berita  Angel Lelga dan Vicky Prasetyo Hebohkan Malam Tahun Baru
masuk
memasukkan berita id  1838389
Insert berita  Ini <i>Lho</i> 3 Cara Cepat Melunasi Utang KPR
masuk
memasukkan berita id  1838563
Insert berita  Yordania Akan Beri Ganjaran Hukum bagi Masyarakat Penyebar <i>Hoax</i> tentang Keluarga Kerajaan
masuk
memasukkan beri

memasukkan berita id  1836828
Insert berita  Mahasiswa Indonesia Boyong 3 Medali Emas dan 4 Penghargaan Tambahan di Taiwan
masuk
memasukkan berita id  1838451
Insert berita  Barcelona Incar Bek Chelsea
masuk
memasukkan berita id  1838461
Insert berita   Masuki 2018, Gaya Menteri Ekonomi Ucapkan Selamat Tahun Baru di Sosmed
masuk
memasukkan berita id  1838465
Insert berita  Kourtney Kardashian Sindir Kylie Jenner yang Absen di Foto Natal Keluarga
masuk
memasukkan berita id  1838520
Insert berita  Lalu Lintas Padat, Tol Jakarta-Cikampek Diberlakukan <i>Contraflow</i> di KM 47-41
masuk
memasukkan berita id  1838516
Insert berita  Choi Bunuh Siswi SMK di Pontianak karena Cemburu
masuk
memasukkan berita id  1838454
Insert berita  Naik 2 Kali Lipat, Ekonomi Singapura Bakal Tumbuh 3,5%   
masuk
memasukkan berita id  1838515
Insert berita  Tren Furnitur 2018 Fokus pada Kenyamanan Penghuni Rumah
masuk
page  6
https://index.okezone.com/bydate/index/2018/01/01/75/
memasukkan berita id  1838514
In

memasukkan berita id  1837394
Insert berita  Ingin Punya Rumah Tahun 2018? Inilah 6 Promo KPR Bunga Paling Murah!
masuk
memasukkan berita id  1838428
Insert berita  Madrid Pecahkan Rekor Transfer demi Hazard
masuk
memasukkan berita id  1838427
Insert berita  Polsek Bontoala Makassar Dibom Molotov, TNI Siap Bantu Polisi Cari Pelaku
masuk
memasukkan berita id  1838426
Insert berita  Penghujung 2017, Sandra Dewi Dikaruniai Anak Pertama
masuk
memasukkan berita id  1838425
Insert berita  Deynica Welirang Wafat, Ini 6 Fakta Anak dan Cucu Pengusaha Sukses Indonesia
masuk
memasukkan berita id  1838424
Insert berita  Demo Besar-besaran, Presiden Iran Peringatkan Warga agar Tak Rusak Fasilitas Publik
masuk
memasukkan berita id  1838423
Insert berita  8 Tips Resolusi Percintaan agar Hubungan Asmara Tetap Membara di 2018
masuk
memasukkan berita id  1838411
Insert berita  MotoGP 2017 Tak Sesuai Ekspektasi Andrea Iannone 
masuk
memasukkan berita id  1838422
Insert berita  Kampanye Celup yang Heboh d

memasukkan berita id  1838163
Insert berita  Ini <i>Game</i> Terbaru yang Bakal Meluncur di 2018
masuk
memasukkan berita id  1838369
Insert berita  Tahun 2018 Pencetakan E-KTP Harus Digenjot 
masuk
memasukkan berita id  1837574
Insert berita  7 Mobil Dalam Film-Film Legendaris   
masuk
memasukkan berita id  1838267
Insert berita  Shahid Kapoor Pamer Foto Keharmonisan dengan Keluarga
masuk
memasukkan berita id  1837654
Insert berita  Dengarkan Musik <i>Up Beat</i>, Cara Sederhana Bangkitkan Semangat Kerja Usai Libur Panjang Akhir Tahun
masuk
page  15
https://index.okezone.com/bydate/index/2018/01/01/210/
memasukkan berita id  1838195
Insert berita  Tahun Anjing Tanah, Sektor Garmen hingga Interior Jadi Peluang Usaha yang Moncer
masuk
memasukkan berita id  1836343
Insert berita  Tips Jalani Masa Peralihan dari Mahasiswa ke Dunia Kerja
masuk
memasukkan berita id  1838227
Insert berita  Bosan di Perjalanan? 4 Buku Ini Bisa Jadi Pilihan Hiburan
masuk
memasukkan berita id  1838331
Insert ber

memasukkan berita id  1838248
Insert berita  James Bond Terbaru Bakal Diperankan oleh Wanita?
masuk
memasukkan berita id  1838324
Insert berita  Kawasan Monas Macet Parah Usai Perayaan Tahun Baru 2018
masuk
memasukkan berita id  1838323
Insert berita  Penembakan Terjadi di Colorado, Seorang Polisi Terbunuh
masuk
memasukkan berita id  1838318
Insert berita  Begini Kemeriahan Suasana Malam Tahun Baru 2018 di Pantai Karnaval Ancol
masuk
memasukkan berita id  1838322
Insert berita  Dentuman Gong Warnai Malam Pergantian Tahun di Kota Solo
masuk
memasukkan berita id  1838230
Insert berita  Bradley Smith: Jadi Pembalap Tim Utama MotoGP Jauh Lebih Sulit
masuk
memasukkan berita id  1838319
Insert berita  Guardiola Bersyukur Man City Raih Satu Poin di Kandang Crystal Palace
masuk
memasukkan berita id  1838320
Insert berita  Begini Suasana Pesta Kembang Api Tahun Baru di Puncak
masuk
memasukkan berita id  1838306
Insert berita  Bersihkan Banten dari Korupsi, Tantangan Gubernur Banten Wahidin Hali

DatabaseError: 1040: Too many connections